In [ ]:
import sys
import re
import requests
import json


def load_log(log_file_path:str = 'test.log', pre_lines:int = 5, post_lines:int = 5):
  with open(log_file_path, 'r') as log_file:
    log_lines = log_file.readlines()

  error_logs = []
  for i, line in enumerate(log_lines):
      if "error" in line.lower():
          start_index = max(0, i - pre_lines)
          end_index = min(len(log_lines), i + post_lines + 1)
          error_logs.extend(log_lines[start_index:end_index])

  return error_logs

error_logs = load_log()

data = {
  "prompt": "\n".join(error_logs), 
  "model": "llama3.1"
}

response = requests.post("http://localhost:11434/api/generate", json=data, stream=True)
for line in response.iter_lines():
  if line:
    json_data = json.loads(line)
    if json_data['done'] == False:
      print(json_data['response'], end='', flush=True)
